# BERT训推全流程实践

本案例通过MindSpore的API来实现BERT的预训练数据构建、模型开发、模型预训练、模型推理全流程实践。

## 环境准备

开发者拿到香橙派开发板后，首先需要进行硬件资源确认、镜像烧录以及CANN和MindSpore版本的升级，才可运行该案例，具体如下：

| 香橙派AIpro | 镜像 | CANN Toolkit/Kernels | MindSpore |
| :----:| :----: | :----:| :----: |
| 20T 24G | Ubuntu | 8.0.0beta1| 2.5.0 |
| 20T 24G | Ubuntu | 8.1RC1beta1| 2.6.0 |

### 镜像烧录

运行该案例需要烧录香橙派官网Ubuntu镜像，参考[镜像烧录](https://www.mindspore.cn/tutorials/zh-CN/r2.6.0rc1/orange_pi/environment_setup.html#1-%E9%95%9C%E5%83%8F%E7%83%A7%E5%BD%95%E4%BB%A5windows%E7%B3%BB%E7%BB%9F%E4%B8%BA%E4%BE%8B)章节。

### CANN升级

参考[CANN升级](https://www.mindspore.cn/tutorials/zh-CN/r2.6.0rc1/orange_pi/environment_setup.html#3-cann%E5%8D%87%E7%BA%A7)章节。

### MindSpore升级

参考[MindSpore升级](https://www.mindspore.cn/tutorials/zh-CN/r2.6.0rc1/orange_pi/environment_setup.html#4-mindspore%E5%8D%87%E7%BA%A7)章节。

In [19]:
import re
from random import *
import mindspore
from mindspore import mint
from mindspore.nn import Cell, Dense
from mindspore.mint import nn, optim
mindspore.set_context(pynative_synchronize=True)

[WARNING] ME(54325:255086409871392,MainProcess):2025-05-16-00:01:47.408.282 [mindspore/context.py:1335] For 'context.set_context', the parameter 'pynative_synchronize' will be deprecated and removed in a future version. Please use the api mindspore.runtime.launch_blocking() instead.


### 预训练任务的数据构建

#### BERT 输入
针对句子对相关任务，将两个句子合并为一个句子对输入到Encoder中，[CLS] + 第一个句子 + [SEP] + 第二个句子 + [SEP];  
针对单个文本相关任务，[CLS] + 句子 + [SEP]。

BERT通过两种无监督任务（Masked Language Modelling 和 Next Sentence Prediction）进行预训练，获取词语和句子级别的特征。
#### Next Sentence Prediction (NSP)
BERT通过NSP捕捉句子级别的信息，使其可以理解句子与句子之间的联系，从而能够应用于问答或者推理任务。
NSP本质上是一个二分类任务，通过输入一个句子对，判断两句话是否为连续句子。

#### Masked Language Model (Masked LM)
BERT模型通过Masked LM捕捉词语层面的信息。  
我们随机将每个句子中15%的词语进行遮盖，替换成掩码\<mask\>。在训练过程中，模型会对句子进行“完形填空”，预测这些被遮盖的词语是什么，通过减小被mask词语的损失值来对模型进行优化。    
由于\<mask\>仅在预训练中出现，为了让预训练和微调中的数据处理尽可能接近，我们在随机mask的时候进行如下操作：
- 80%的概率替换为\<mask\>
- 10%的概率替换为文本中的随机词
- 10%的概率不进行替换，保持原有的词元

In [20]:
# sample IsNext and NotNext to be same in small batch size
def make_batch():
    batch = []
    positive = negative = 0 # 为了记录NSP任务中正样本和负样本的个数，比例最好是在一个batch中接近1:1
    while positive != batch_size/2 or negative != batch_size/2:
        tokens_a_index, tokens_b_index = randrange(len(sentences)), randrange(len(sentences)) # sample random index in sentences
        tokens_a, tokens_b = token_list[tokens_a_index], token_list[tokens_b_index]
        input_ids = [word_dict['[CLS]']] + tokens_a + [word_dict['[SEP]']] + tokens_b + [word_dict['[SEP]']]
        segment_ids = [0] * (1 + len(tokens_a) + 1) + [1] * (len(tokens_b) + 1)

        # MASK LM
        n_pred =  min(max_pred, max(1, int(round(len(input_ids) * 0.15)))) # 15 % of tokens in one sentence
        cand_maked_pos = [i for i, token in enumerate(input_ids)
                          if token != word_dict['[CLS]'] and token != word_dict['[SEP]']]
        shuffle(cand_maked_pos)
        masked_tokens, masked_pos = [], []
        for pos in cand_maked_pos[:n_pred]:
            masked_pos.append(pos)
            masked_tokens.append(input_ids[pos])
            if random() < 0.8:  # 80%
                input_ids[pos] = word_dict['[MASK]'] # make mask
            elif random() < 0.5:  # 10%
                index = randint(0, vocab_size - 1) # random index in vocabulary
                input_ids[pos] = word_dict[number_dict[index]] # replace

        # Zero Paddings
        n_pad = maxlen - len(input_ids)
        input_ids.extend([0] * n_pad)
        segment_ids.extend([0] * n_pad)

        # Zero Padding (100% - 15%) tokens
        if max_pred > n_pred:
            n_pad = max_pred - n_pred
            masked_tokens.extend([0] * n_pad)
            masked_pos.extend([0] * n_pad)

        if tokens_a_index + 1 == tokens_b_index and positive < batch_size/2:
            batch.append([input_ids, segment_ids, masked_tokens, masked_pos, 1]) # IsNext
            positive += 1
        elif tokens_a_index + 1 != tokens_b_index and negative < batch_size/2:
            batch.append([input_ids, segment_ids, masked_tokens, masked_pos, 0]) # NotNext
            negative += 1
    return batch
# Proprecessing Finished

### get_attn_pad_mask
是为了得到句子中pad的位置信息，给到模型后面，在计算自注意力和交互注意力的时候去掉pad符号的影响

比如说，我现在的句子长度是5，在后面注意力机制的部分，我们在计算出来QK转置乘以根号之后，softmax之前，我们得到的形状len_input*len_input ,代表每个单词对其余包含自己的单词的影响力   

所以这里需要一个同等大小形状的矩阵，告诉我哪个部分是pad部分，之后在计算softmax之前会把这里置为负无穷大 一定要注意的是这里得到的矩阵形状是batch_size x len_q x len_k，我们是对K中的pad符号进行标识并没有对Q中的做标识，因为没有必要。

In [21]:
def get_attn_pad_mask(seq_q, seq_k):
    batch_size, len_q = seq_q.shape
    batch_size, len_k = seq_k.shape
    
    # pad_attn_mask = ops.equal(seq_k, 0)
    pad_attn_mask = mint.eq(seq_k, 0)
    pad_attn_mask = pad_attn_mask.expand_dims(1) # batch_size x 1 x len_k(=len_q), one is masking

    # return ops.broadcast_to(pad_attn_mask, (batch_size, len_q, len_k)) # batch_size x len_q x len_k
    return mint.broadcast_to(pad_attn_mask, (batch_size, len_q, len_k)) # batch_size x len_q x len_k

### BERT Embedding
输入到BERT模型的信息由三部分内容组成：

- 表示内容的token ids  
- 表示位置的position ids  
- 用于区分不同句子的token type ids

三种信息分别进入Embedding层，得到token embeddings、position embeddings与segment embeddings；与Transformer不同，以上三种均为可学习的信息。

In [22]:
class BertEmbedding(Cell):
    def __init__(self):
        super(BertEmbedding, self).__init__()
        self.tok_embed = mindspore.nn.Embedding(vocab_size, d_model, dtype=mindspore.float16)  # token embedding
        self.pos_embed = mindspore.nn.Embedding(maxlen, d_model, dtype=mindspore.float16)  # position embedding
        self.seg_embed = mindspore.nn.Embedding(n_segments, d_model, dtype=mindspore.float16)  # segment(token type) embedding
        self.norm = mindspore.nn.LayerNorm([d_model,], epsilon=1e-7, dtype=mindspore.float16)

    def construct(self, x, seg):
        # MindSpore中表示Tensor形状的属性为shape，区别于PyTorch中的size
        seq_len = x.shape[1]
        pos = mint.arange(seq_len, dtype=mindspore.int32)
        pos = pos.expand_dims(0).expand_as(x)  # (seq_len,) -> (batch_size, seq_len)
        embedding = self.tok_embed(x) + self.pos_embed(pos) + self.seg_embed(seg)
        return self.norm(embedding)

### 注意力机制

相同的一句话，不同的人听的时候侧重点也可能不同。在自然语言处理中，根据任务内容的不同，句子中需要重点关注的部分也会不同。为此，我们引入注意力机制来判断在执行某个任务时，词在句子中的重要性，并通过注意力分数来表示词的重要程度。分数越高，说明该词对完成该任务的重要性越大。
在计算注意力分数时，我们主要参考三个因素：**query**、**key**和**value**。

- `query`：任务内容
- `key`：索引/标签（帮助定位到答案）
- `value`：答案

注意力分数的计算公式为：

$$\text{Attention Score}(Q, K)=\frac{QK^T}{\sqrt{d_{model}}}$$

同时，为了避免`query`（$Q \in R^{n\times d_{model}}$）和`key`($K \in R^{m\times d_{model}}$)本身的“大小”影响到相似度的计算，我们需要在点乘后除以$\sqrt{d_{model}}$。

$$\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_{model}}}\right)V$$

在如下代码中，我们实现了scaled dot-product attention的计算， 调用类后，返回的是加权后的value（context）以及注意力权重（attn）。

In [23]:
# 基于mindspore.ops手动实现softmax
def manual_softmax(x, dim=-1, dtype=mindspore.float16):
    exp_x = mindspore.ops.exp(x - mindspore.ops.max(x, axis=dim, keepdims=True)[0])
    return (exp_x / mindspore.ops.sum(exp_x, dim=dim, keepdim=True)).to(dtype)

In [24]:
class ScaledDotProductAttention(Cell):
    def __init__(self):
        super(ScaledDotProductAttention, self).__init__()

    def construct(self, Q, K, V, attn_mask):
        scores = mint.matmul(Q, K.swapaxes(-1, -2)) / mint.sqrt(mindspore.ops.scalar_to_tensor(d_k, mindspore.float16)) # scores : [batch_size x n_heads x len_q(=len_k) x len_k(=len_q)]
        scores = scores.masked_fill(attn_mask, -1e9) # Fills elements of self tensor with value where mask is one.
        attn = manual_softmax(scores, dim=-1, dtype=mindspore.float16)
        context = mint.matmul(attn, V)
        return context, attn

### 多头注意力（Multi-Head Attention）

多头注意力是注意力机制的扩展，它可以使模型通过不同的方式关注输入序列的不同部分，从而提升模型的训练效果。

不同于之前一次计算整体输入的注意力分数，多头注意力是多次计算，每次计算输入序列中某一部分的注意力分数，最后再将结果进行整合。

多头注意力通过对输入的embedding乘以不同的权重参数$W^{Q}$、$W^{K}$和$W^{V}$，将其映射到多个小维度空间中，我们称之为“头”（head），每个头部会并行计算自己的自注意力分数。

$$\text{head}_i = \text{Attention}(XW^Q_i, XW^K_i, XW^V_i) = \text{softmax}\left(\frac{Q_iK_i^T}{\sqrt{d_{k}}}\right)V_i$$

$W^Q_i \in \mathbb{R}^{d_{model}\times d_{k}}$、$W^K_i \in \mathbb{R}^{d_{model}\times d_{k}}$和$W^V_i \in \mathbb{R}^{d_{model}\times d_{v}}$为可学习的权重参数。一般为了平衡计算成本，我们会取$d_k = d_v = d_{model} / n_{head}$。

在获得多组自注意力分数后，我们将结果拼接到一起，得到多头注意力的最终输出。$W^O$为可学习的权重参数，用于将拼接后的多头注意力输出映射回原来的维度。

$$\text{MultiHead}(Q, K, V)=\text{Concat}(\text{head}_1, ..., \text{head}_h)W^O$$

简单来说，在多头注意力中，每个头部可以'解读'输入内容的不同方面，比如：捕捉全局依赖关系、关注特定语境下的词元、识别词和词之间的语法关系等。

### Add & Norm

Add & Norm层本质上是残差连接后紧接了一个LayerNorm层。

$$\text{AddNorm}(x) = \text{LayerNorm}(x + \text{Sublayer}(x))$$

- Add：残差连接，帮助缓解网络退化问题，注意需要满足$x$与$\text{SubLayer}(x)的形状一致$；
- Norm：Layer Norm，层归一化，帮助模型更快地进行收敛；

#### MindSpore与PyTorch的全连接层API对比

MindSpore中构造全连接层的API为nn.Dense，区别于PyTorch中的nn.Linear，且二者权重初始化不同：

- MindSpore：weight默认初始化为'normal'分布，bias默认初始化为零
- PyTorch：weight默认初始化为kaiming_uniform，bias默认初始化为uniform

In [25]:
class MultiHeadAttention(Cell):
    def __init__(self):
        super(MultiHeadAttention, self).__init__()
        self.W_Q = Dense(d_model, d_k * n_heads, dtype=mindspore.float16)
        self.W_K = Dense(d_model, d_k * n_heads, dtype=mindspore.float16)
        self.W_V = Dense(d_model, d_v * n_heads, dtype=mindspore.float16)
        self.W_O = Dense(n_heads * d_v, d_model, dtype=mindspore.float16)
        self.attn = ScaledDotProductAttention()
        self.norm = mindspore.nn.LayerNorm([d_model,], epsilon=1e-7, dtype=mindspore.float16)

    def construct(self, Q, K, V, attn_mask):
        # q: [batch_size x len_q x d_model], k: [batch_size x len_k x d_model], v: [batch_size x len_k x d_model]
        residual, batch_size = Q, Q.shape[0]
        # (B, S, D) -proj-> (B, S, D) -split-> (B, S, H, W) -trans-> (B, H, S, W)
        # 注意区分MindSpore中的Tensor.transpose与PyTorch中的Tesor.transpose
        # MindSpore中的transpose为对所有维度进行重排，PyTorch中的transpose为对两个维度进行交换
        # MindSpore中对Tensor的两个维度进行交换的接口应为swapaxes
        q_s = self.W_Q(Q).view(batch_size, -1, n_heads, d_k).swapaxes(1,2)  # q_s: [batch_size x n_heads x len_q x d_k]
        k_s = self.W_K(K).view(batch_size, -1, n_heads, d_k).swapaxes(1,2)  # k_s: [batch_size x n_heads x len_k x d_k]
        v_s = self.W_V(V).view(batch_size, -1, n_heads, d_v).swapaxes(1,2)  # v_s: [batch_size x n_heads x len_k x d_v]
        
        attn_mask = attn_mask.expand_dims(1)
        attn_mask = mint.tile(attn_mask, (1, n_heads, 1, 1))
        
        # context: [batch_size x n_heads x len_q x d_v], attn: [batch_size x n_heads x len_q(=len_k) x len_k(=len_q)]
        context, attn = self.attn(q_s, k_s, v_s, attn_mask)
        context = context.swapaxes(1, 2).view(batch_size, -1, n_heads * d_v) # context: [batch_size x len_q x n_heads * d_v]
        output = self.W_O(context)
        return self.norm(output + residual), attn # output: [batch_size x len_q x d_model]

### 基于位置的前馈神经网络 （Position-Wise Feed-Forward Network）
基于位置的前馈神经网络被用来对输入中的每个位置进行非线性变换。它由两个线性层组成，层与层之间需要经过GELU激活函数。

FFN(𝑥)=GELU(𝑥𝑊1+𝑏1)𝑊2+𝑏2
 
相比固定的GELU函数，基于位置的前馈神经网络可以处理更加复杂的关系，并且由于前馈网络是基于位置的，可以捕获到不同位置的信息，并为每个位置提供不同的转换。

#### MindSpore与PyTorch的GELU激活函数对比

对于API的功能对齐，注意MindSpore和PyTorch中有些API即使名称宏观功能一致，默认传参会出现差别，如GELU接口。

- MindSpore：入参`approximate`为`boolean`类型，默认为True，如果为True采取tanh方法做优化近似，反之采取erf方法
- PyTorch：入参`approximate`为`string`类型，默认为'none'，如果为'tanh'采取tanh方法进行优化近似

In [26]:
class PoswiseFeedForwardNet(Cell):
    def __init__(self):
        super(PoswiseFeedForwardNet, self).__init__()
        self.fc1 = Dense(d_model, d_ff, dtype=mindspore.float16)
        self.fc2 = Dense(d_ff, d_model, dtype=mindspore.float16)
        self.activation = nn.GELU()

    def construct(self, x):
        # (batch_size, len_seq, d_model) -> (batch_size, len_seq, d_ff) -> (batch_size, len_seq, d_model)
        return self.fc2(self.activation(self.fc1(x)))

### Encoder Layer
我们首先实现encoder中的一个层。

In [27]:
class EncoderLayer(Cell):
    def __init__(self):
        super(EncoderLayer, self).__init__()
        self.enc_self_attn = MultiHeadAttention()
        self.pos_ffn = PoswiseFeedForwardNet()

    def construct(self, enc_inputs, enc_self_attn_mask):
        # print('enc_inputs: ', enc_inputs.dtype)
        enc_inputs = enc_inputs.astype(mindspore.float16)
        enc_outputs, attn = self.enc_self_attn(enc_inputs, enc_inputs, enc_inputs, enc_self_attn_mask) # enc_inputs to same Q,K,V
        enc_outputs = self.pos_ffn(enc_outputs) # enc_outputs: [batch_size x len_q x d_model]
        return enc_outputs, attn


### BERT

将上面实现的encoder层堆叠`n_layers`次，并添加token embeddings、position embeddings与segment embeddings。

#### BERT 输出
BERT会针对每一个位置输出大小为hidden size的向量，在下游任务中，会根据任务内容的不同，选取不同的向量放入输出层。

我们一般称[CLS]经过线性层+激活函数tanh的输出为pooler output，用于句子级别的分类/回归任务;  
我们一般称BERT输出的每个位置对应的vector为sequence output,用于词语级别的分类任务；

In [28]:
class BERT(Cell):
    def __init__(self):
        super(BERT, self).__init__()
        self.embedding = BertEmbedding()
        self.layers = mindspore.nn.CellList([EncoderLayer() for _ in range(n_layers)])
        self.fc = Dense(d_model, d_model, dtype=mindspore.float16)
        self.activ1 = nn.Tanh()
        self.linear = Dense(d_model, d_model, dtype=mindspore.float16)
        self.activ2 = nn.GELU()
        self.norm = mindspore.nn.LayerNorm([d_model,], epsilon=1e-7, dtype=mindspore.float16)
        self.classifier = Dense(d_model, 2, dtype=mindspore.float16)
        # decoder is shared with embedding layer
        embed_weight = self.embedding.tok_embed.embedding_table
        n_vocab, n_dim = embed_weight.shape
        self.decoder = Dense(n_dim, n_vocab, has_bias=False, dtype=mindspore.float16)
        self.decoder.weight = embed_weight.to(mindspore.float16)
        self.decoder_bias = mindspore.Parameter(mint.zeros(n_vocab, dtype=mindspore.float16), 'decoder_bias')

    def construct(self, input_ids, segment_ids, masked_pos):
        output = self.embedding(input_ids, segment_ids)
        enc_self_attn_mask = get_attn_pad_mask(input_ids, input_ids)
        for layer in self.layers:
            output, enc_self_attn = layer(output, enc_self_attn_mask)
        h_pooled = self.activ1(self.fc(output[:, 0]))  # [batch_size, d_model]
        logits_clsf = self.classifier(h_pooled)  # [batch_size, 2]
        
        # 使用ops.gather替代gather_elements，无需扩展masked_pos为三维

        h_masked = mindspore.ops.gather(output, masked_pos, 1, batch_dims=1)  # [batch_size, max_pred, d_model]

        h_masked = self.norm(self.activ2(self.linear(h_masked)))
        logits_lm = self.decoder(h_masked) + self.decoder_bias  # [batch_size, max_pred, n_vocab]


        return logits_lm, logits_clsf

In [29]:
# BERT Parameters
maxlen = 30 # maximum of length
batch_size = 6
max_pred = 5  # max tokens of prediction
n_layers = 6 # number of Encoder of Encoder Layer
n_heads = 12 # number of heads in Multi-Head Attention
d_model = 768 # Embedding Size
d_ff = 768 * 4  # 4*d_model, FeedForward dimension
d_k = d_v = 64  # dimension of K(=Q), V
n_segments = 2

### 构建词典
将每个词元映射到数字索引中，词元和数字索引所构成的集合叫做词典（vocabulary）。

在构建词典中，我们使用了4个特殊词元。
[PAD]：填充词元（padding），当句子长度不够时将句子填充至统一长度；  
[CLS]：句子级别信息；  
[SEP]: 结束词元，用来标注一个句子的结束；  
[MASK]: mask词元；


In [ ]:
text = (
    'Hello, how are you? I am Romeo.\n'
    'Hello, Romeo My name is Juliet. Nice to meet you.\n'
    'Nice meet you too. How are you today?\n'
    'Great. My baseball team won the competition.\n'
    'Oh Congratulations, Juliet\n'
    'Thanks you Romeo'
)
sentences = re.sub("[.,!?\\-]", '', text.lower()).split('\n')  # filter '.', ',', '?', '!'
word_list = list(set(" ".join(sentences).split()))
word_dict = {'[PAD]': 0, '[CLS]': 1, '[SEP]': 2, '[MASK]': 3}
for i, w in enumerate(word_list):
    word_dict[w] = i + 4
number_dict = {i: w for i, w in enumerate(word_dict)}
vocab_size = len(word_dict)
token_list = list()
for sentence in sentences:
    arr = [word_dict[s] for s in sentence.split()]
    token_list.append(arr)

### 模型训练
实例化BERT模型。

### 定义损失函数与优化器。

- 损失函数：定义如何计算模型输出(logits)与目标(targets)之间的误差，这里可以使用交叉熵损失（CrossEntropyLoss）  
- 优化器：MindSpore将模型优化算法的实现称为优化器。优化器内部定义了模型的参数优化过程（即梯度如何更新至模型参数），所有优化逻辑都封装在优化器对象中。

In [31]:
model = BERT()
criterion = mindspore.ops.SoftmaxCrossEntropyWithLogits()
optimizer = mindspore.nn.Adam(model.trainable_params(), learning_rate=1e-4)

### 模型训练逻辑

MindSpore在模型训练中采取函数式编程，即：

1. Network+loss function直接构造正向函数
2. 函数变换，获得梯度计算（反向传播）函数
3. 构造训练过程函数
4. 调用函数进行训练

和PyTorch相比，两个框架在前向计算的调用方式一致，但在反向计算中MindSpore采用函数微分，PyTorch采用数据流微分，最后在梯度更新中，二者流程相同，但底层实现逻辑不同。


In [ ]:
from mindspore import amp

loss_scaler = amp.StaticLossScaler(scale_value=2**10)

# 构造正向函数
def forward(input_ids, segment_ids, masked_pos, masked_tokens, isNext):
    logits_lm, logits_clsf = model(input_ids, segment_ids, masked_pos)
    logits_lm = mint.reshape(logits_lm, (-1, vocab_size)) # [batch_size * max_pred, n_vocab]
    masked_tokens = mint.reshape(masked_tokens, (-1,)) # [batch_size * max_pred]
    masked_tokens_onehot = nn.functional.one_hot(masked_tokens.astype(mindspore.int32), vocab_size)
    
    loss_lm = criterion(logits_lm, masked_tokens_onehot.to(logits_lm.dtype))[0]
    loss_lm = loss_lm.mean()
    # scaled_loss_lm = loss_scaler.scale(loss_lm)

    isNext_onehot = nn.functional.one_hot(isNext.astype(mindspore.int32), 2)
    loss_clsf = criterion(logits_clsf, isNext_onehot.to(logits_clsf.dtype))[0]
    loss_clsf = loss_clsf.mean()
    # scaled_loss_clsf = loss_scaler.scale(loss_clsf)

    return loss_scaler.scale(loss_lm + loss_clsf)


In [34]:
# 函数变换，获得微分函数
grad_fn = mindspore.value_and_grad(forward, None, optimizer.parameters)

MindSpore提供了jit装饰器，可以通过修饰Python函数或者Python类的成员函数，使其被编译成计算图，通过图优化等技术提高运行速度。在这种方式下，我们可以简单地对想要进行性能优化的模块来进行图编译加速，而模型的其他部分仍旧使用解释执行方式，不丢失动态图的灵活性。

In [35]:
# 定义单步训练逻辑

def train_step(input_ids, segment_ids, masked_pos, masked_tokens, isNext):
    # 反向传播获取梯度
    loss, grads = grad_fn(input_ids, segment_ids, masked_pos, masked_tokens, isNext)
    unscaled_loss = loss_scaler.unscale(loss)
    unscaled_grads = loss_scaler.unscale(grads)
    # 模型权重更新
    optimizer(unscaled_grads)
    return unscaled_loss


In [36]:
# 构建数据
batch = make_batch()
input_ids, segment_ids, masked_tokens, masked_pos, isNext = map(mindspore.Tensor, zip(*batch))

# 启动模型训练
import time
model.set_train()
for epoch in range(2):
    print('Epoch:', '%04d' % (epoch + 1))
    s_time = time.time()
    loss = train_step(input_ids, segment_ids, masked_pos, masked_tokens, isNext)
    print('loss:', loss.asnumpy())

    e_time = time.time()
    print('time:', e_time - s_time)
    # loss = train_step(input_ids, segment_ids, masked_pos, masked_tokens, isNext) # for sentence classification
    if (epoch + 1) % 1 == 0:
        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss.asnumpy()))

Epoch: 0001
.....loss: 3.855
time: 74.99734854698181
Epoch: 0001 cost = 3.855469
Epoch: 0002
loss: 2.363
time: 0.5860943794250488
Epoch: 0002 cost = 2.363281


### 以batch中的第一组语句batch[0]为例，进行测试。

In [37]:
# Predict mask tokens ans isNext
batch = make_batch()
input_ids, segment_ids, masked_tokens, masked_pos, isNext = map(mindspore.Tensor, zip(batch[0]))
print(f"文本内容：\n {text}")
print("掩码之后的文本：")
print([number_dict[int(w.asnumpy())] for w in input_ids[0] if number_dict[int(w.asnumpy())] != '[PAD]'])


logits_lm, logits_clsf = model(input_ids, segment_ids, masked_pos)
logits_lm_reshaped = mint.reshape(logits_lm, (-1, vocab_size)) # [batch_size * max_pred, n_vocab]
masked_tokens_reshaped = mint.reshape(masked_tokens, (-1,)) # [batch_size * max_pred]
masked_tokens_onehot = nn.functional.one_hot(masked_tokens_reshaped.astype(mindspore.int32), vocab_size)
loss_lm = criterion(logits_lm_reshaped, masked_tokens_onehot.to(logits_lm.dtype))[0]

loss_lm = loss_lm.mean()

logits_lm = logits_lm.argmax(2)[0].asnumpy()

print('masked tokens list : ', [pos for pos in masked_tokens[0] if pos != 0])
print('predict masked tokens list : ', [pos for pos in logits_lm if pos != 0])

logits_clsf = logits_clsf.argmax(1).asnumpy()[0]

print('isNext : ', True if isNext else False)
print('predict isNext : ',True if logits_clsf else False)

文本内容：
 Hello, how are you? I am Romeo.
Hello, Romeo My name is Juliet. Nice to meet you.
Nice meet you too. How are you today?
Great. My baseball team won the competition.
Oh Congratulations, Juliet
Thanks you Romeo
掩码之后的文本：
['[CLS]', 'hello', 'romeo', '[MASK]', 'name', 'is', 'juliet', 'nice', 'to', 'meet', 'you', '[SEP]', '[MASK]', 'you', 'romeo', '[SEP]']
masked tokens list :  [Tensor(shape=[], dtype=Int64, value= 6), Tensor(shape=[], dtype=Int64, value= 7)]
predict masked tokens list :  []
.isNext :  False
predict isNext :  True
